## Downloading required dataset:

In [175]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip -d Collection5
!rm -rf collection5.zip
!pip install corus
!pip install natasha
!wget https://github.com/dialogue-evaluation/factRuEval-2016/archive/master.zip
!unzip master.zip
!rm master.zip

--2024-07-09 14:12:35--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.08MB/s    in 0.9s    

2024-07-09 14:12:37 (2.08 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/Collection5/
  inflating: Collection5/Collection5/001.ann  
  inflating: Collection5/Collection5/001.txt  
  inflating: Collection5/Collection5/002.ann  
  inflating: Collection5/Collection5/002.txt  
  inflating: Collection5/Collection5/003.ann  
  inflating: Collection5/Collection5/003.txt  
  inflating: Collection5/Collection5/004.ann  
  inflating: Collection5/Collection5/004.txt  
  inflating: Collection5/Collection5/005.ann  
  infl

## Importing libraries

In [176]:
from corus import load_ne5, load_factru
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    Doc
)
import pandas as pd
from sklearn.metrics import f1_score

## Initialize dataset + needded functions

In [212]:
dir = 'Collection5/Collection5'
dir2 = 'factRuEval-2016-master/'
records2 = load_factru(dir2)
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
records = load_ne5(dir)
segmenter = Segmenter()

## Filing predicted + true labels

In [178]:
def solving(record, id):
  predicted_labels = []
  true_labels = []
  for j in range(1000):
    text = next(record)
    doc = Doc(text.text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for i in range(len(text.spans)):
        true_labels.append(text.spans[i].type)
        if i < len(doc.ner.spans):
            if (doc.spans[i].type == 'LOC'):
              predicted_labels.append('GEOPOLIT')
            else:
              predicted_labels.append(doc.spans[i].type)
        else:
            predicted_labels.append('wrong')

  return true_labels, predicted_labels

In [179]:
def solving2(record, id):
  try:
    predicted_labels = []
    true_labels = []
    for _ in range(1000):
      text = next(record)
      doc = Doc(text.text)
      check = 0
      doc.segment(segmenter)
      doc.tag_ner(ner_tagger)
      for j in range(len(text.objects)):
        for i in range(len(text.objects[j].spans)):
          true_labels.append(text.objects[j].spans[i].type)
          if check < len(doc.ner.spans):
              if (doc.spans[check].type == 'LOC'):
                predicted_labels.append('loc_name')
              elif (doc.spans[check].type == 'ORG'):
                  predicted_labels.append('org_name')
              elif (doc.spans[check].type == 'PER'):
                  predicted_labels.append('name')
              else:
                predicted_labels.append(doc.spans[check].type)
          else:
              predicted_labels.append('wrong')
          check += 1
  except StopIteration:
    pass
  return true_labels, predicted_labels

## Check f1-score

In [180]:
def check_score(record, id):
  if id == 0:
    true_labels, predicted_labels = solving(record, id)
  else:
    true_labels, predicted_labels = solving2(record, id)
  f1_scores = f1_score(true_labels, predicted_labels, average='weighted')
  print(f"F1-Score: {f1_scores:.4f}")

## Factrueval



In [181]:
check_score(records2, 1)

F1-Score: 0.1879


## Collection5

In [182]:
check_score(records, 0)


F1-Score: 0.5692
